# Tokenize pre-mRNA sequences

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
import multiprocessing as mp
import pandas as pd
import random
import os

/opt/anaconda3/envs/torch201/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
os.chdir('/picb/rnasys2/zhouhanwen/nucTran/github/')

In [3]:
# tokenizer
tokenizer_path = 'tokenizer/single_nucleotide/'
model_max_length = 1026
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, model_max_length=model_max_length)

In [4]:
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["seq"], truncation=True, max_length=tokenizer.model_max_length)
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs

In [5]:
# paths of training set and testing set
train_set_path = 'SpliceSitePrediction/data/SpliceAI/training/trainset_split_768.json'
# path of saving file
data_path = 'SpliceSitePrediction/data/SpliceAI/training/ss_single_nucleotide'

In [6]:
# This step takes ~10 mins. Please wait a moment.
dataset = load_dataset("json", data_files=train_set_path)
data = dataset.map(group_texts, batched=True, num_proc=mp.cpu_count())
data = data.remove_columns(["seq", "label"])
data = data["train"].train_test_split(test_size=0.001, shuffle=True, seed=42)
data.save_to_disk(data_path, num_proc=mp.cpu_count())

Found cached dataset json (/home/zhouhanwen/.cache/huggingface/datasets/json/default-dab8c03657eb5336/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/zhouhanwen/.cache/huggingface/datasets/json/default-dab8c03657eb5336/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-dc97da9d36cb8398_*_of_00160.arrow


Saving the dataset (0/160 shards):   0%|          | 0/949608 [00:00<?, ? examples/s]

Saving the dataset (0/160 shards):   0%|          | 0/951 [00:00<?, ? examples/s]

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 949608
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 951
    })
})